In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://calendar.google.com/calendar/u/0/embed?src=7a04205ae91b85e82ebc74daddbf2933c6b6723b81abb966f0e69c66a996c43b@group.calendar.google.com&ctz=America/Vancouver&pli=1).**

## Imports

In [2]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
from gensim.models import KeyedVectors

def find_similar_words(word, top_n=5):
    similar_words = glove_wiki_vectors.most_similar(word, topn=top_n)
    return [word for word, _ in similar_words]

target_word = 'dog'
similar_words = find_similar_words(target_word)

print(f"Similar words to '{target_word}':")
for word in similar_words:
    print(word)

Similar words to 'dog':
cat
dogs
pet
puppy
horse


In [7]:
def find_similar_words(word, top_n=5):
    similar_words = glove_wiki_vectors.most_similar(word, topn=top_n)
    return [word for word, _ in similar_words]

target_word = 'delicious'
similar_words = find_similar_words(target_word)

print(f"Similar words to '{target_word}':")
for word in similar_words:
    print(word)

Similar words to 'delicious':
tasty
spicy
dessert
savory
refreshing


In [8]:
def find_similar_words(word, top_n=5):
    similar_words = glove_wiki_vectors.most_similar(word, topn=top_n)
    return [word for word, _ in similar_words]

target_word = 'music'
similar_words = find_similar_words(target_word)

print(f"Similar words to '{target_word}':")
for word in similar_words:
    print(word)

Similar words to 'music':
musical
songs
dance
pop
recording


In [9]:
def find_similar_words(word, top_n=5):
    similar_words = glove_wiki_vectors.most_similar(word, topn=top_n)
    return [word for word, _ in similar_words]

target_word = 'adventure'
similar_words = find_similar_words(target_word)

print(f"Similar words to '{target_word}':")
for word in similar_words:
    print(word)

Similar words to 'adventure':
adventures
fantasy
trek
fiction
horror


In [10]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings
rubric={autograde:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
coast_shore_similarity = None
clothes_closet_similarity = None
old_new_similarity = None
smart_intelligent_similarity = None
dog_cat_similarity = None
tree_lawyer_similarity = None

coast_shore_similarity = glove_wiki_vectors.similarity("coast", "shore")
clothes_closet_similarity = glove_wiki_vectors.similarity("clothes", "closet")
old_new_similarity = glove_wiki_vectors.similarity("old", "new")
smart_intelligent_similarity = glove_wiki_vectors.similarity("smart", "intelligent")
dog_cat_similarity = glove_wiki_vectors.similarity("dog", "cat")
tree_lawyer_similarity = glove_wiki_vectors.similarity("tree", "lawyer")

print("Cosine Similarity:")
print(f"Coast-Shore: {coast_shore_similarity}")
print(f"Clothes-Closet: {clothes_closet_similarity}")
print(f"Old-New: {old_new_similarity}")
print(f"Smart-Intelligent: {smart_intelligent_similarity}")
print(f"Dog-Cat: {dog_cat_similarity}")
print(f"Tree-Lawyer: {tree_lawyer_similarity}")

Cosine Similarity:
Coast-Shore: 0.7000271677970886
Clothes-Closet: 0.5462760329246521
Old-New: 0.6432487368583679
Smart-Intelligent: 0.7552732825279236
Dog-Cat: 0.8798075318336487
Tree-Lawyer: 0.07671945542097092


In [13]:
grader.check("q1.2")

q1.2 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [14]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [15]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [16]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [17]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 6

In [18]:
analogy("man", "programmer", "woman")

man : programmer :: woman : ?


,Analogy word,Score
0,educator,0.585333
1,programmers,0.573053
2,linguist,0.543201
3,technician,0.537336
4,freelance,0.536212
5,animator,0.533937
6,translator,0.533419
7,software,0.496662
8,psychotherapist,0.495662
9,technologist,0.488051


In [19]:
analogy("man", "breadwinner", "woman")

man : breadwinner :: woman : ?


,Analogy word,Score
0,caregiver,0.647082
1,breadwinners,0.609576
2,carer,0.583033
3,homemaker,0.557918
4,helpmate,0.544433
5,spouse,0.529185
6,widowed,0.526016
7,step-mother,0.524767
8,prioress,0.521728
9,daughter-in-law,0.518864


In [20]:
glove_wiki_vectors.similarity("black", "crime")

0.43576175

In [21]:
glove_wiki_vectors.similarity("white", "crime")

0.38129756

In [22]:
glove_wiki_vectors.similarity("caucasian", "short")

0.1332649

In [23]:
glove_wiki_vectors.similarity("indian", "short")

0.40793177

We can see that the word embeddings are very stereotypical and biased. In the examples above we can see that a man is to programmer and breadwinner as a woman is to educator and caregiver. These roles are very stereotypical gender roles. Sometimes these biases can get problematic. We can see above that the similarity scores for black and crime are higher than the the similarity score for white and crime. Similarly there is a higher similarity of indian and short versus caucasian and short.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

1. Sometimes, the data may be statistically accurate, but that may be due to societal systemic biases. This is at no fault of the embeddings, but rather the data that it is trained on/present. It is of high quality, but based on the data it was trained on it could be problematic. 
2. If word embeddings with gender bias are used in decision-making systems, such as in automated hiring or resume screening, they can lead to unfair and discriminatory outcomes. Biases present in the embeddings can influence the automated systems' judgments and perpetuate gender-based discrimination in employment and other domains.

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 
rubric={autograde:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [24]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [25]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [26]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [27]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()
from sklearn.metrics import classification_report

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [28]:
bow_train_accuracy = None
bow_test_accuracy = None
bow_report = None # use digits=2

from sklearn.preprocessing import StandardScaler

vectorizer = CountVectorizer(stop_words="english")

X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

logreg_bow = LogisticRegression(max_iter=1000)
logreg_bow.fit(X_train_bow, y_train)

bow_train_accuracy = logreg_bow.score(X_train_bow, y_train)
bow_test_accuracy = logreg_bow.score(X_test_bow, y_test)

y_pred_bow = logreg_bow.predict(X_test_bow)
bow_report = classification_report(y_test, y_pred_bow, digits=2)
print(bow_train_accuracy)
print(bow_test_accuracy)
print(bow_report)

0.9562051178314959
0.8173666823973572
                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238



In [29]:
from math import floor

X_train_embeddings = None
X_test_embeddings = None

nlp = spacy.load("en_core_web_md")

def calculate_embeddings(texts):
    embeddings = []
    for text in texts:
        doc = nlp(text)
        text_embedding = doc.vector.reshape(1, -1)
        embeddings.append(text_embedding)
    return np.concatenate(embeddings, axis=0)

X_train_embeddings = calculate_embeddings(X_train)
X_test_embeddings = calculate_embeddings(X_test)

logreg_emb = LogisticRegression(max_iter=2000)
logreg_emb.fit(X_train_embeddings, y_train)

emb_train_accuracy = logreg_emb.score(X_train_embeddings, y_train)
emb_test_accuracy = logreg_emb.score(X_test_embeddings, y_test)

y_pred_emb = logreg_emb.predict(X_test_embeddings)
emb_report = classification_report(y_test, y_pred_emb, digits=2)

/Applications/anaconda3/envs/cpsc330/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
grader.check("q1.5")

q1.5 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

_Points:_ 6

1. CountVectorizer is a technique that converts text documents into a matrix of word counts, treating each word as a separate feature. It focuses on word frequencies and ignores word order. The average-embedding approach, on the other hand, represents text by averaging the word embeddings of individual words, capturing semantic relationships and contextual information. It considers the meaning and context of words rather than just their frequencies.
2. The CountVectorizer representation tends to be more interpretable compared to the average-embedding representation. This is because it directly captures the occurrence and frequency of individual words in the text, allowing for a straightforward understanding of which words are present and how often they appear. This representation can be easily interpreted by examining the feature weights or coefficients associated with each word, providing insights into the importance of specific words in the classification task.
3. Yes, the use of pre-trained word embeddings such as en_core_web_md in the average-embedding approach involves transfer learning.The benefits of transfer learning in this context are that the pre-trained word embeddings provide a rich representation of word meanings and contextual relationships. This helps capture the semantic nuances of the text, even with limited training data. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [31]:
from sklearn.datasets import fetch_20newsgroups

In [32]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [33]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [34]:
import re
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

if 'sentencizer' not in nlp.pipe_names:
    nlp.add_pipe('sentencizer')

def preprocess_text(text):
    text = re.sub(r"[\\/]", " ", text)

    doc = nlp(text)
    sentences = list(doc.sents)

    sentences = [re.sub(r"\b[\w\-.]+?@\w+?\.\w{2,4}\b", "", str(sentence)) for sentence in sentences]
    sentences = [re.sub(r"http\S+|www\S+|\S+\.com\S+", "", str(sentence)) for sentence in sentences]

    sentences = [re.sub(r"\b\w{1,2}\b", "", str(sentence)) for sentence in sentences]

    processed_sentences = []
    for sentence in sentences:
        tokens = [token.lemma_.lower() for token in nlp(sentence) if not token.is_stop and not token.is_punct]
        processed_sentences.append(" ".join(tokens))
    
    return " ".join(processed_sentences)

df["text_pp"] = df["text"].apply(preprocess_text)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

Removing the slashes or weird characters helps in cleaning the text and removing irrelevant symbols that don't contribute to the topics.
Sentence segmentation and tokenization allows for a detailed analysis by breaking the text into individual sentences and words.
Eliminating URLs and email addresses helps in removing specific information that is not relevant to the overall topics.
Removing fairly unique tokens helps in reducing noise and focusing on more meaningful and frequent terms.
Excluding stopwords and punctuation improves the quality of topic modeling by filtering out commonly occurring and less informative words.
Lemmatization reduces word variations, ensuring that words with the same base meaning are treated as a single entity during topic modeling.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

In [35]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_2 = CountVectorizer()
X = vectorizer_2.fit_transform(df["text_pp"])

n_topics = 7  
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)

lda.fit(X)

LatentDirichletAllocation(n_components=7, random_state=42)

When determining the number of topics, it's important to consider the complexity and diversity of the dataset. In this case, we're working with a subset of scikit-learn's 20 newsgroups dataset, which consists of texts from different categories such as sports, religion, and politics.

One approach is to consider the number of categories in the dataset. In this case, there are 8 categories. Hence, setting the number of topics to 8 seems reasonable as it aligns with the known categories and can potentially capture the main themes in the data. However, while testing, we found that 7 topics has the best outcome. We saw that there was an overlap in the top words and themes between two topics sometimes (eg: 'don', 'know') and so we thought it might be more appropriate to combine them into a single category to avoid redundancy and improve clarity.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [36]:
feature_names = vectorizer_2.get_feature_names()
topics = []
for topic_idx, topic in enumerate(lda.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    topics.append((topic_idx, top_words))


for topic_idx, top_words in topics:
    print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")

Topic 1: file, window, use, program, entry, widget, server, include, run, edu
Topic 2: armenian, turkish, armenians, people, jews, turkey, greek, turks, government, genocide
Topic 3: god, people, believe, think, jesus, know, question, say, don, mean
Topic 4: gun, state, file, weapon, law, firearm, control, right, government, israel
Topic 5: know, say, don, people, think, go, like, time, come, didn
Topic 6: game, team, play, year, player, win, season, good, hockey, league
Topic 7: image, edu, file, know, graphic, thank, program, jpeg, color, use


/Applications/anaconda3/envs/cpsc330/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


1. Software Development
2. Religious Politics
3. Religion
4. Firearms
5. Common words
6. Sports
7. Graphic design

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>
_Points:_ 5

In [37]:
X = vectorizer_2.transform(df["text_pp"])

document_topic_assignment = lda.transform(X[:5])

for doc_idx, document in enumerate(document_topic_assignment):
    top_topic_idx = document.argmax()
    top_topic = topics[top_topic_idx][0]
    print(f"Document {doc_idx+1}: Topic {top_topic}")

Document 1: Topic 3
Document 2: Topic 6
Document 3: Topic 6
Document 4: Topic 0
Document 5: Topic 1


The document topic assignment indicates the most likely topic for each of the first five documents in the dataset. Each document is assigned to one of the identified topics based on the content of the document.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1.

- Decision Tree: It handles multiclass classification naturally. The algorithm splits the feature space into regions based on the class labels and creates decision rules to assign class labels to new instances. It can handle both binary and multiclass classification problems without any specific modifications.

- KNN: It handles multiclass classification by using the "one-vs-one" or "one-vs-all" strategies. In the "one-vs-one" strategy, separate binary classifiers are trained for each pair of classes, and majority voting is used for prediction. In the "one-vs-all" strategy, a separate binary classifier is trained for each class against all the other classes, and the class with the highest probability is predicted.

- Random Forest: It also handles multiclass classification naturally. The algorithm combines multiple decision trees and uses either the "one-vs-one" or "one-vs-all" strategies internally to handle multiclass problems. Each decision tree in the Random Forest independently predicts the class label, and the final prediction is made based on majority voting among the trees.

- Logistic Regression: This handles multiclass classification using the "one-vs-rest" strategy. In the "one-vs-rest" strategy, a separate binary logistic regression model is trained for each class against all the other classes. Each model predicts the probability of belonging to its respective class, and the class with the highest probability is predicted.

- SVM RBF: This uses the "one-vs-one" strategy. Separate binary SVM models are trained for each pair of classes, and majority voting is used for prediction. The RBF kernel allows SVMs to handle non-linear decision boundaries, which can be effective for complex multiclass classification problems.

2. Transfer learning refers to utilizing pre-trained models trained on large datasets to extract useful features or representations. These models capture contextual information and visual patterns, respectively. By leveraging these pre-trained models, new tasks benefit from the learned representations even with limited task-specific data. The pre-trained models can be fine-tuned on smaller datasets to adapt to the specific task, improving performance. Transfer learning saves time and computational resources, addresses data scarcity, and enables faster convergence. However, the effectiveness depends on the similarity between the pre-training and target tasks/domains. 

3. It allows for the reuse and modification of different parts of the network. In transfer learning, the pre-trained layers of a neural network, known as the "base network," capture general features and patterns from a large-scale dataset. By treating the pre-trained layers as a fixed feature extractor and appending new layers for the specific target task, only the new layers need to be trained on the task-specific data. This approach enables the transfer of knowledge from the pre-trained layers to the new layers, resulting in improved performance with limited labeled data. The sequential nature of neural networks facilitates this modular transfer and adaptation of learned representations, making transfer learning effective and efficient.

<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)